# Basic tutorial: translation
#### Author: Matteo Caorsi

This short tutorial provides you with the basic functionalities of *giotto-deep* API.

The example described in this tutorial is the one of translation following a sequence-to-sequence transformer model: all the details can be found [here](https://papers.nips.cc/paper/2017/file/3f5ee243547dee91fbd053c1c4a845aa-Paper.pdf). 

## Probabilistic modelling

The basic idea would be to predict the target sentence $y$ given source sentence $x$, more precisely you want to predict $P(y|x)$. But this is very hard to do directly, since one would need a lot of data to have a reasonable estimate of this conditional probability.
Hence, one would rather factor the probability using the chain rule, then predict each factor:

$$
P(y_1,...,y_N|x) = \prod_i^N P(y_i|y_{<i},x)
$$

In the NLP language, this corresponds to predicting the next token of the output sequence given the already predicted tokens and the input sequence.

## Scope

You will build your own transformer model, train it, and use it to translate German sentences to English!

Here is an example of what a translation task is about:

 - German sentence: 'Ich mag Pizza.'
 - English translation: 'I like pizza.'

## The plan for this tutorial

The main steps of the tutorial are the following:
 1. Creation of a dataset
 2. Preprocessing of the dataset
 3. Creation of a model
 4. Definition of the metrics and losses
 5. Trainining of the model
 6. Using the model to translate some sentences
 7. (extra) Extract some features of the network for interpretability analysis
 
Let's start with importing the required libraries!

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import copy
import sys

import numpy as np
import torch
from torch import nn
from gtda.diagrams import BettiCurve
from gtda.plotting import plot_betti_surfaces
from torchtext.data.utils import get_tokenizer
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn import Transformer
from torch.optim import Adam, SparseAdam, SGD
import torchtext

# our special guests!
from gdeep.models import FFNet
from gdeep.visualization import persistence_diagrams_of_activations
from gdeep.data.datasets import DatasetBuilder
from gdeep.trainer import Trainer
from gdeep.data import TransformingDataset
from gdeep.data.preprocessors import TokenizerTranslation
from gdeep.data.datasets import DataLoaderBuilder
from gdeep.models import ModelExtractor
from gdeep.analysis.interpretability import Interpreter
from gdeep.visualization import Visualiser
from gdeep.search import GiottoSummaryWriter


No TPUs...


# Initialize the tensorboard writer

In order to analyse the results of your models, you need to start tensorboard. All data about the model, the training, the hyperparameters... will be stored there.

## How to start tensorboard
On the terminal, move inside the `/examples` folder. There run the following command:

```
tensorboard --logdir=runs
```

Then go [here](http://localhost:6006/) after the training step to visualise all the results.

In [2]:
writer = GiottoSummaryWriter()


# Create your dataset

We propose to use the [Multi30k](https://github.com/multi30k/dataset) dataset: in giotto-deep it is very easy to import datasets! It is enough to write:

```python
# initialise the builder with the dataset name
bd = DatasetBuilder(name="Multi30k", convert_to_map_dataset=True)

# build the datasets
ds_tr_str, ds_val_str, ds_ts_str = bd.build()

```

### Disclaimer: the next cell is a temporary fix to a third-party download link issue.

In [3]:
# temporary cell, due to a failure of third party link: we are required to download the dataset manually!
# yout would expect to simply run:
# bd = DatasetBuilder(name="Multi30k", convert_to_map_dataset=True)

# and then
# ds_tr_str, ds_val_str, ds_ts_str = bd.build()

# instead we are doing this mess:

import os
import gzip
import shutil
from typing import List

from torchdata.datapipes.iter import FileOpener, FileLister, StreamReader
from torch.utils.data import Dataset

from gdeep.utility import DEFAULT_DOWNLOAD_DIR


base_path = os.path.join(DEFAULT_DOWNLOAD_DIR, "Multi30k")
url_de = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/train.de.gz'
url_en = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/train.en.gz'
torchtext.utils.download_from_url(url_en, path=os.path.join(base_path, "train.en.gz"))
torchtext.utils.download_from_url(url_de, path=os.path.join(base_path, "train.de.gz"))
url_de = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/val.de.gz'
url_en = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/val.en.gz'
torchtext.utils.download_from_url(url_en, path=os.path.join(base_path, "val.en.gz"))
torchtext.utils.download_from_url(url_de, path=os.path.join(base_path, "val.de.gz"))
url_de = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/test_2016_flickr.de.gz'
url_en = 'https://github.com/multi30k/dataset/raw/master/data/task1/raw/test_2016_flickr.en.gz'
torchtext.utils.download_from_url(url_en, path=os.path.join(base_path, "test.en.gz"))
torchtext.utils.download_from_url(url_de, path=os.path.join(base_path, "test.de.gz"))

# we store the data in the RAM
in_memory_data = []


for item in ["train.de", "train.en", "val.de", "val.en", "test.de", "test.en"]:
    with gzip.open(os.path.join(base_path, item+".gz"), 'rb') as f_in:
        with open(os.path.join(base_path, item), 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)

    with open(os.path.join(base_path, item), 'r', encoding="utf8") as f:
        in_memory_data.append(f.read())

assert len(in_memory_data) == 6
assert len(in_memory_data[0].split("\n")) == len(in_memory_data[1].split("\n"))
assert len(in_memory_data[2].split("\n")) == len(in_memory_data[3].split("\n"))

class MyMulti30k(Dataset):
    """Temporary Multi30k Dataset."""

    def __init__(self, data_tuple: List[List[str]]) -> None:
        """
        Args:
            data:
                list of data
        """
        
        assert len(data_tuple) == 2, "expected 2 lists corresponding to the two languages"
        self.data = list(zip(data_tuple[0].split("\n"), data_tuple[1].split("\n")))


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    
ds_tr_str, ds_val_str, ds_ts_str = MyMulti30k(in_memory_data[:2]), MyMulti30k(in_memory_data[2:4]), MyMulti30k(in_memory_data[4:6])

The dataset contains a list of pairs of sentences: the German sentence and its English translation.

In [4]:
print("Before preprocessing: \n", ds_tr_str[0])


Before preprocessing: 
 ('Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.', 'Two young, White males are outside near many bushes.')


## Required preprocessing

Neural networks cannot direcly deal with strings. We have first to preprocess the dataset in three main ways:
 1. Tokenise each string into its words (and maybe adjust each word to remove plurals, interjections, capital letters...)
 2. Build a vocabulary out of these tokens (each modified word of point a. is called a token)
 3. Embed each token into a vector, so that each sentence becomes a list of vectors

The **first two steps** are performed by the `TokenizerTranslation` class. The embedding will be added directly to the model (see below).

In [5]:

# install german tokenizer
!{sys.executable} -m spacy download de_core_news_sm

# get the german tokenizer
de_tokenizer = get_tokenizer('spacy', language='de_core_news_sm')

# initialise the giotto-deep tokenizer
tokenizer = TokenizerTranslation(tokenizer=de_tokenizer)

# fit the tokenizer to the dataset (note that te vocabularies will be automatically built in this case
tokenizer.fit_to_dataset(ds_tr_str)

# prprocess the dataset
transformed_textds = tokenizer.attach_transform_to_dataset(ds_tr_str)
transformed_textts = tokenizer.attach_transform_to_dataset(ds_val_str)

print("After the preprocessing: \n", transformed_textds[0])

# subsample the training and test datasets
train_indices = list(range(64*2))
test_indices = list(range(64*1))

dl_tr, dl_val, _ = DataLoaderBuilder((transformed_textds, 
                                      transformed_textts)).build(({"batch_size":16, 
                                                                   "sampler":SubsetRandomSampler(train_indices)},{"batch_size":16, 
                                                                                                                  "sampler":SubsetRandomSampler(test_indices)}
                                                                 ))


/home/mehdi/Documents/giotto-deep/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 33.8 MB/s eta 0:00:00m eta 0:00:010:01:01
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


/home/mehdi/Documents/giotto-deep/venv/lib/python3.8/site-packages/torch/cuda/__init__.py:497: UserWarning:

Can't initialize NVML



After the preprocessing: 
 ([tensor([  18,   82,  254,   28,   84,   19,   91,    4,   13,  109, 5494, 3158,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0]), tensor([  13,   20,   12,   21,  766,   14,   53,   76,  194, 1297,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0])], tensor([  13,   20,   12,   21,  766,   14,   53,   76,  194, 1297,    2,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0]))


As you can see, now the data is not in string format anymore: rather, the sentences have each been transformed to a `torch.Tensor` of type `long`. Each of these numbers represent the index in the vocabulary of the associated token.

## Define your model

The model we play with is a simple transformer model with two embedding layers (for the German and English sentence) followed by a single transformer layer. 

The input of this model is assumed to be a tensors. The tensor is of the form `shape(transformed_textds[0][0])`. 

Of course, as in all neural networks, data are inputted to the model  in batches: hence, the first dimension will be the batch size, then a dimension to choose the source or target sentence, and the last dimension will be about the actual tokenisation.

In [6]:
# my simple transformer model
class TranslationTransformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, embed_dim):
        super(TranslationTransformer, self).__init__()
        self.transformer = Transformer(
            d_model=embed_dim,
            nhead=2,
            num_encoder_layers=1,
            num_decoder_layers=1,
            dim_feedforward=512,
            dropout=0.1,
        )
        self.embedding_src = nn.Embedding(src_vocab_size, embed_dim, sparse=True)
        self.embedding_tgt = nn.Embedding(tgt_vocab_size, embed_dim, sparse=True)
        self.generator = nn.Linear(embed_dim, tgt_vocab_size)

    def forward(self, src, tgt):
        src_emb = self.embedding_src(src).permute(1, 0, 2)
        tgt_emb = self.embedding_tgt(tgt).permute(1, 0, 2)
        self.outs = self.transformer(src_emb, tgt_emb).permute(1, 0, 2)
        logits = self.generator(self.outs)
        return logits

    def encode(self, src, src_mask):
        """this method is used only at the inference step.
        This method runs the data through the encoder of the
        transformer"""
        return self.transformer.encoder(self.embedding_src(src), src_mask)

    def decode(self, tgt, memory, tgt_mask):
        """this method is used only at the inference step
        This method runs the data through the decoder of the
        transformer"""
        return self.transformer.decoder(self.embedding_tgt(tgt), memory, tgt_mask)


We initialise the model as follows:
 - we need to set the maximum vocabulary size to fix the `Embedding` architectures
 - we need to set the embedding dimension
 - initialise the model class with the needed parameters.

In [7]:
src_vocab_size = len(tokenizer.vocabulary)
tgt_vocab_size = len(tokenizer.vocabulary_target)
emb_dim = 64

model = TranslationTransformer(src_vocab_size, tgt_vocab_size, emb_dim)
X = next(iter(dl_tr))
# a datum
# assert model(X[0]).argmax(2).shape == X[1].shape
print("This is our model: \n", model)


This is our model: 
 TranslationTransformer(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=64, out_features=64, bias=True)
          )
          (linear1): Linear(in_features=64, out_features=512, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=512, out_features=64, bias=True)
          (norm1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0): TransformerDecoderLayer(
          (self_attn): Multihead

## Define the loss function

This loss function is an adapted version of the Cross Entropy Loss for the transformer architecture we just defined.

In [8]:
def loss_fn(logits, tgt_out):
    cel = nn.CrossEntropyLoss()
    return cel(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))


And here we are: all is set up and we are ready to train the model.

# Trainig the model

In giotto deep all is done via a `Trainer`, meaning a class that takes care of the training and validation steps, storing the intermediate results to tensorboard, using (or not) cross validation, ... basically everything you need.

In [9]:
# prepare a pipeline class with the model, dataloaders loss_fn and tensorboard writer
pipe = Trainer(model, (dl_tr,), loss_fn, writer)

# train the model
pipe.train(
    optimizer=SGD,
    n_epochs=2,
    cross_validation=False,
    optimizers_param={"lr": 0.01},
    dataloaders_param={"batch_size": 16},
)


Epoch 1
-------------------------------
IN INNER TRAINING LOOP
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207]) 
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207]) 
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207]) 
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])   
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])     
AAAAAAAAAAAH torch.Size([6, 44]) torch.Size([6, 40]) torch.Size([6, 40]) torch.Size([6, 40, 10207])         
Epoch training loss: 8.055317 	Epoch training accuracy: 954.46%                                            
Time taken for this epoch: 0.00s
Learning rate value: 0.01000000
DANS VALIDATION
TA M

/home/mehdi/Documents/giotto-deep/gdeep/trainer/trainer.py:668: UserWarning:

Cannot store data in the PR curve



AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])    
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])     
AAAAAAAAAAAH torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])    
AAAAAAAAAAAH torch.Size([6, 44]) torch.Size([6, 40]) torch.Size([6, 40]) torch.Size([6, 40, 10207])         
Epoch training loss: 5.753362 	Epoch training accuracy: 2706.25%                                           
Time taken for this epoch: 0.00s
Learning rate value: 0.01000000
DANS VALIDATION
TA MERE
JE PELURE torch.Size([16, 44]) torch.Size([16, 40]) torch.Size([16, 40]) torch.Size([16, 40, 10207])
aLED2 torch.Size([16, 1, 10207])
JE PELURE torch.Size([10, 44]) torch.Size([10, 40]) torch.Size([10, 40]) torch.Size([10, 40, 10207])
aLED2 torch.Size([10, 1, 10207])
Validation results: 
 accuracy: 2687.50%,                 Avg loss: 2.166215 



(2.16621470451355, 2687.5)

## Translation!

So the model has been trained and now it is time to translate a sentence. Let's take the following example:

In [10]:
de, en = next(iter(ds_tr_str))
print(de, "\n", en)


Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche. 
 Two young, White males are outside near many bushes.



Get the vocabulary and numericize the German sentence

In [11]:
voc = tokenizer.vocabulary
# sent = str.lower(de).split()
# de_sentence = list(map(voc.__getitem__,sent))
de_sentence, en_sentence = tokenizer((de, en))[0]
de_sentence


tensor([  18,   82,  254,   28,   84,   19,   91,    4,   13,  109, 5494, 3158,
           1,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0])

We need to build a couple of auxiliary functions to help us translate. The logic is to use the transformer encoder and decoder layers directly. Let's have a quick reminder of how things work:

<img src="./images/translation_transformer.png" alt="drawing" width="400" class="center"/>

The above drawing is a good representation of our architecture. During training, the English and German sentence are inputted from "below". Then, once the model is used in inference, there is an important change ongoing. Basically, the full input sentence (the German one in our case) is given as input to the encoder: the output (the probablities) are then greedily transformed into a single token: that token is then used in the "Outputs" below: together with the Input sentence, the next word is predicted (at the top) and used again as input in the "Outputs" below. Hence, word after word, you get the translation of the original sentence.

In [12]:
from gdeep.utility import DEVICE

pad_item = 0
max_len = len(en_sentence)


def greedy_decode(model, src, src_mask, max_len):
    """function to generate output sequence using greedy algorithm"""

    memory = model.encode(src, src_mask)
    # ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long)
    ys = torch.ones(*src.shape).fill_(pad_item).type(torch.long).to(DEVICE)
    for i in range(max_len - 1):
        # print("before:", ys)
        out = model.decode(ys, memory, None)
        out = out.transpose(0, 1)
        prob = model.generator(out[:, -1])
        next_word = torch.argmax(prob, dim=1)
        next_word = next_word[i + 1].item()  # the shift
        ys[0, i] = next_word
        # print("after:", ys)
    return ys


def translate(model: torch.nn.Module, dl_ts_item):
    """actual function to translate input sentence into target language"""
    model.eval()
    voc = tokenizer.vocabulary_target
    src = dl_ts_item.to(DEVICE)
    num_tokens = src.shape[1]
    src_mask = None
    tgt_tokens_tensor = greedy_decode(model, src, src_mask, max_len).flatten().to(DEVICE)
    tgt_tokens = tgt_tokens_tensor.tolist()
    # print(tgt_tokens)
    return " ".join(voc.lookup_tokens(tgt_tokens))


# translation!
print("German sentence: ", de)
print("English translation: ", translate(pipe.model, de_sentence.reshape(1, -1)))


German sentence:  Zwei junge weiße Männer sind im Freien in der Nähe vieler Büsche.
English translation:  <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk>


## Extra: interpretability

In the next section we show you how to run some simple intepretability tools, like plotting the attention matrices of the transformer. 

The first step is to extract the values of the attention activations given an input `x`.

In [13]:
from gdeep.models import ModelExtractor

# the candidate datum
x = [de_sentence.view(1,-1).to(DEVICE), en_sentence.view(1,-1).to(DEVICE)]  # only one pair of sentences, shapd as a batch of dim = 1

# the model extractor
ex = ModelExtractor(pipe.model, loss_fn)

# getting the names of the layers
layer_names = ex.get_layers_param().keys()

print("Let's extract the activations of the first attention layer: ", next(iter(layer_names)))
self_attention = ex.get_activations(x)[:2]


Let's extract the activations of the first attention layer:  transformer.encoder.layers.0.self_attn.in_proj_weight


In [14]:
# let's plot the tensor! First, load th visualizer
vs = Visualiser(pipe)
vs.plot_self_attention(self_attention, tokenizer.tokenizer(en), de_tokenizer(de), figsize=(20, 20));


If you have just run this notebooks, you would most likely have gotten some funny nonsensical answer: consider that you trained a very simple model on a very small subset of data for only two epochs.

## Challenge

Starting from this simple notebook, do you think you can enlarge the dataset, the model, the training epochs and get a decent translator? Good luck!

# (WIP) Huggingface transformers

If you are actually interested in a pretrained transformers that works directly with a few lines of code, `giotto-deep` supports Hugginface transformers.

The next section explains how to run one such transformer.

In [15]:
# from transformers import ...

# take T5

# finetune on the Multi30k

# go to TB to see the results + screenshot
